In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import spherical_jn, lpmv


# -------------------------------------------------------
# Paramètres physiques
# -------------------------------------------------------
f = 40000          # fréquence (Hz)
c0 = 343           # vitesse du son dans l'air (m/s)
ω = 2 * np.pi * f
k = ω / c0         # nombre d'onde

A = 1.0            # amplitude du vortex
l = 6       # ordre du vortex (radial)
m = 1              # moment angulaire (nombre topologique)
t = 0              # temps instantané pour la visualisation


# -------------------------------------------------------
# Espace de simulation
# -------------------------------------------------------
N =8 # nombre de transducteur par axe (8x8)
a=0.01

# bornes symétriques : +/-5 cm latéralement, 8 cm en hauteur
Lx = 0.05
Ly = 0.05
DzT=0.08

x = np.linspace(-Lx, Lx, N)
y = np.linspace(-Ly, Ly, N)

X_T_b, Y_T_b = np.meshgrid(x, y)
Z_T_b=np.array([[0]*N]*N)

X_T_h, Y_T_h = np.meshgrid(x, y)
Z_T_h=np.array([[DzT]*N]*N)

##on passe tout à une dimension, dans le ref de la plaque :
X_T_b=X_T_b.flatten()
Y_T_b=Y_T_b.flatten()
Z_T_b=Z_T_b.flatten()

X_T_h=X_T_h.flatten()
Y_T_h=Y_T_h.flatten()
Z_T_h=Z_T_h.flatten()

X_T=np.concatenate((X_T_b,X_T_h))
Y_T=np.concatenate((Y_T_b,Y_T_h))
Z_T=np.concatenate((Z_T_b,Z_T_h))

# ---- Constantes physiques ----
k = 2 * np.pi * 40000 / 343
R_p = 0.0005
c_p = 2400
rho_p = 28.59
c_0 = 343
rho_0 = 1.204
R_t = 0.005  # rayon transducteur

eps = 1e-4  # pas de dérivation

In [2]:
def compute_vortex_phases(xf, yf, zf):

    ## dans le ref du vortex :

    Z_T_v=Z_T-zf
    X_T_v=X_T-xf
    Y_T_v=Y_T-yf

    # -------------------------------------------------------
    # Passage en coordonnées sphériques
    # -------------------------------------------------------
    r = np.sqrt(X_T_v**2 + Y_T_v**2 + Z_T_v**2)
    θ = np.arccos(Z_T_v/ (r + 1e-12))  # angle polaire
    φ = np.arctan2(Y_T_v, X_T_v)            # angle azimutal

    # -------------------------------------------------------
    # Calcul du champ acoustique vortex(r,θ,φ)
    # -------------------------------------------------------
    jl = spherical_jn(l, k * r)
    Plm = lpmv(m, l, np.cos(θ))

    vortex = A * jl * Plm * np.exp(1j * (m * φ - ω * t))

    # -------------------------------------------------------
    # Amplitude et phase
    # -------------------------------------------------------
    ampl = np.abs(vortex)
    phase = np.angle(vortex)
    phase=np.mod(phase,2*np.pi)

    # normalisation amplitude (comme tu fais déjà)
    ampl = ampl / np.max(ampl)

    return ampl, phase

In [3]:
# === Point de focalisation ===
X_foc, Y_foc, Z_foc = 0, 0, 0.03

# === Lancement ===
compute_vortex_phases(X_foc, Y_foc, Z_foc)

(array([0.01677714, 0.0267962 , 0.04636474, 0.02848555, 0.01443892,
        0.05840451, 0.01635536, 0.0843237 , 0.0843237 , 0.01635536,
        0.05840451, 0.01443892, 0.02848555, 0.04636474, 0.0267962 ,
        0.01677714, 0.0267962 , 0.01060969, 0.00151039, 0.05840451,
        0.14562036, 0.04102542, 0.23681151, 0.31107823, 0.31107823,
        0.23681151, 0.04102542, 0.14562036, 0.05840451, 0.00151039,
        0.01060969, 0.0267962 , 0.04636474, 0.00151039, 0.03499666,
        0.13834916, 0.31107823, 0.13058881, 0.46898256, 0.37518039,
        0.37518039, 0.46898256, 0.13058881, 0.31107823, 0.13834916,
        0.03499666, 0.00151039, 0.04636474, 0.02848555, 0.05840451,
        0.13834916, 0.27344789, 0.46898256, 0.27922044, 0.54273055,
        0.25099052, 0.25099052, 0.54273055, 0.27922044, 0.46898256,
        0.27344789, 0.13834916, 0.05840451, 0.02848555, 0.01443892,
        0.14562036, 0.31107823, 0.46898256, 0.54278347, 0.1761516 ,
        0.43217076, 0.14231051, 0.14231051, 0.43